<a href="https://colab.research.google.com/github/juanchess98/Notebooks-Deep-Learning/blob/Recurrent-Neural-Network/Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales Recurrentes - Recurrent Neural Networks (RNN's)
Una red neuronal recurrente se parece mucho a una red neuronal convencional con la excepción de que también tiene conexiones apuntando hacia atrás.
La RNN más simple está compuesta de una neurona que recibe entradas y produce una salida y envía la salida de vuelta a si misma. En cada time step $t$   (también llamado frame), esta neurona recurrente recibe las entradas $\mathbf{x}_{(t)}$ así como su propio output del time step anterior, $\mathbf{y}_{(t-1)}$.
Ya que no hay un output anterior en el primer time step, este se asigna generalmente el valor de 0. Esta pequeña red se puede representar contra el eje del tiempo. Esto se conoce como desenrrollar la red en el tiempo(unrolling the network through time).
![RNN](https://drive.google.com/uc?id=1zOI-isp2DN-42ZFjtBKK1XFXDkSi3FiM)

Tu puedes crear facilmente un layer de neuronas recurrentes. En cada time step $t$, cada neurona recibe el vector de entrada $\mathbf{x}_{(t)}$ y el output del vector del time step anterior $\mathbf{y}_{(t-1)}$. Note que las entradas y las salidas son vector ! 

Cada neurona recurrente tiene dos pesos. Uno para las entradas $\mathbf{x}_{(t)}$ y otro para las salidas del time step anterior $\mathbf{y}_{(t-1)}$ denotados como las matrices $\mathbf{W}_{x}$ y $\mathbf{W}_{y}$ respectivamente.

La salida de una capa recurrente para una sola instancia está dada así:
$$\mathbf{y}_{(t)} = \phi(\mathbf{W}_{x}^{T}\mathbf{x}_{(t)} +\mathbf{W}_{y}^{T}\mathbf{y}_{(t-1)} + \mathbf{b})$$
$\mathbf{b}$ es el vector bias y $\phi(.)$ es la función de activación(ReLu o tanh son las más utilizadas).

Asi como en las redes neuronales vistas anteriormente, se puede computar la salida de una capa de una capa recurrente de una sola vez al colocar todas las entradas en el time step t en una matriz de entrada $\mathbf{X}_{(t)}$ así:

$$\mathbf{Y}_{(t)} = \phi(\mathbf{W}_{x}^{T}\mathbf{X}_{(t)} +\mathbf{W}_{y}^{T}\mathbf{Y}_{(t-1)} + \mathbf{b})$$



## Memory Cells

Ya que la salida de una neurona recurrente en cada time step es una función de todas las entradas de los time step anteriores se podría decir que tiene una forma de memoria. Una parte de una red neuronal que preserva algún tipo de estado a lo largo de los time steps es llamada una memory cell(o simplemente una cell). A una simple neurona recurrente o una capa de neuronas recurrentes es una cell muy básica, capaz de de aprender patrones cortos(tipicamente de un largo de 10 steps, pero varía de acuerdo a la tarea).

De manera general, el estado de una celda en el time step $t$, denotado como $\mathbf{h}_{(t)}$(la "h" es por "hidden"), es una función de algunas entradas en ese time step y de su estado en el time step anterior: $\mathbf{h}_{(t)} = f(\mathbf{h}_{(t-)}, \mathbf{x}_{(t)})$. Su salida en el time step $t$, denotada como $\mathbf{y}_{(t)}$ es también una función del estado previo y las entradas del time step actuales. En el caso las celdas básicas que hemos discutido hasta ahora la salida es simplemente igual al estado, pero en celdas más complejas este no es siempre el caso. La salida en este caso es: $\mathbf{Y}_{(t)} = \mathbf{W}_{ht}\mathbf{h}_{(t)}$ una matriz de pesos que multiplica al estado actual. 

![RNN with output different](https://drive.google.com/uc?id=1dGUf3itlQnss6TkQ3sR9W6q-1oDtoZDY)


## Secuencias de Entrada y Salida

Una RNN toma una secuencia de entrada y la convierte en una secuencia de salida. Este tipo de red llamada sequence-to-sequence es útil para predecir series de tiempo tal  como stock prices.

Alternativamente, tu podrías ignorar todas las salidas excepto la última. En otras palabras, esto es una sequence-to-vector network. Por ejemplo, tu podrías alimentar la red con una sequencia de palabras correspondientes a la opinión sobre una pelicula y la salida de la red seria un puntaje de sentimiento(de -1[hate] hasta +1[love]).

Contrariamente, tu puedes darle a la red el mismo vector de entrada una y otra vez en cada time step y dejas como salida una sequencia. Este tipo de red se denomina vector-to-sequence. Un ejemplo de esto podría ser una imagen(o la salida de una CNN) a la entrada de la red y la salida podría ser una descripción de esa imagen.

Por último, tu podrías tener una red sequence-to-vector llamada encoder, seguido de una red vector-to-sequence llamada decoder. Por ejemplo, esto podría ser utilizado para traducir una oración de un lenguaje a otro 
![Types of Networks](https://drive.google.com/uc?id=1R34_wnKdzhfflNbBe9cVkf2cAmy3p_eh)


In [1]:
import numpy as np

In [4]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
  return series[..., np.newaxis].astype(np.float32)

Esta función crea cuantas series de tiempo como sea requerido(a través del batch_size argument). Cada una de estas con una longitud de n_steps y hay solo un valor por cada time step en cada serie. Esto significa que todas las series son univariadas.
Cuando se trabaja con series de tiempo(y otro tipos de secuencias como oraciones), las características de entradas son representadas generalmente como un vector de 3 Dimensiones con la forma [batch size, time steps, dimensionality], donde la dimensionality es 1 para series de tiempo univariadas y más de 1 para series de tiempo multivariadas.

In [5]:
# Creemos un set de entrenamiento, validación y test:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1] # 
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1] 


X_train contiene 7000 series de tiempo. Su forma es [7000, 50, 1], mientras que X_valid contiene 2000 y X_test 1000


In [7]:
print(f"El shape de X_train es: {X_train.shape}")
print(f"El shape de X_train es: {X_valid.shape}")
print(f"El shape de X_train es: {X_test.shape}")

El shape de X_train es: (7000, 50, 1)
El shape de X_train es: (2000, 50, 1)
El shape de X_train es: (1000, 50, 1)


Antes de empezar a utilizar RNNs es a menudo unas métricas de base o sino podemos pensar que nuestro modelo está trabajando bien cuando de hecho lo está haciendo peor que los modelos básicos. Por ejemplo, la forma más simple es predecir el último valor en cada serie. Esto se denomina naive forecasting, y es algunas veces sorprendentemente dificil de superar. 

In [8]:
from tensorflow import keras


In [21]:
y_pred = X_valid[:, -1]
mse= np.mean(keras.losses.mean_absolute_error(y_valid, y_pred))
print("Mean squared error {:.4f}".format(mse))

Mean squared error 0.1152


Miremos si podemos vencer eso con una RNN simple.

In [20]:
model = keras.models.Sequential([
keras.layers.SimpleRNN(1, input_shape=[None, 1])])

Esta es realmente la RNN más simple que puedes construir. Esta contiene un solo layer con una sola neurona. No necesitamos especificar la longitud de las secuencias de entrada porque una red neuronal recurrente puede procesar cualquier número de time steps(Por esto es que colocamos la primera dimension de entrada igual a None). La cada SimpleRNN usa de manera predeterminada la función de activación tangente hiperbólica.

In [25]:
 optimizer = keras.optimizers.Adam(learning_rate=0.001) # Optimizador 
 model.compile(optimizer=optimizer ,loss='mse',metrics=['mse']) # Funcion de costo a optimizar y optimizador

In [26]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 4s 11ms/step - loss: 0.0583 - mse: 0.0583 - val_loss: 0.0455 - val_mse: 0.0455
Epoch 2/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0390 - mse: 0.0390 - val_loss: 0.0306 - val_mse: 0.0306
Epoch 3/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.0204 - val_mse: 0.0204
Epoch 4/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0178 - mse: 0.0178 - val_loss: 0.0147 - val_mse: 0.0147
Epoch 5/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.0123 - val_mse: 0.0123
Epoch 6/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.0115 - val_mse: 0.0115
Epoch 7/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 8/20
219/219 [==============================] - 2s 10ms/step - 

Como se puede ver el MSE alcanza solo 0.010 así que es mejor el modelo base. Otro punto a favor de una RNN, es que en una RNN simple, el número de parametros es el número de neuronas recurrentes en una capa más el término bias.

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
